<a href="https://colab.research.google.com/github/MeeGyo/dataviz2025/blob/main/midterm2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# นางสาวภาวิณี หน่อสีดา 663020288-2

ให้สืบหาฆาตรกร (10 คะแนน) และ ผู้บงการ (5 คะแนน) ในคดี ฆาตรกรรม (murder) ที่เกิดขึ้นในวันที่ 15 มกราคม 2018 ที่เมือง Pandas City โดยเริ่มจากการตรวจสอบ ข้อมูลคดีในไฟล์ crime_scene_report.csv

In [ ]:
import os
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
base_path = "/content/drive/MyDrive/midtermdataviz"

In [ ]:
crime = pd.read_csv(base_path + "/crime_scene_report.csv")
interview = pd.read_csv(base_path + "/interview.csv")
person = pd.read_csv(base_path + "/person.csv")
drivers = pd.read_csv(base_path + "/drivers_license.csv")
fb_checkin = pd.read_csv(base_path + "/facebook_event_checkin.csv")
fit_checkin = pd.read_csv(base_path + "/get_fit_now_check_in.csv")
fit_member = pd.read_csv(base_path + "/get_fit_now_member.csv")
income = pd.read_csv(base_path + "/income.csv")

##หาพยาน

In [77]:
murder_report = crime[(crime['date'] == 20180115) &
                      (crime['type'] == 'murder') &
                      (crime['city'] == 'Pandas City')]
murder_report


,date,type,description,city
1227,20180115,murder,Security footage shows that there were 2 witne...,Pandas City


In [29]:
#พยานคนแรก
northwestern_residents = person[person['address_street_name'] == 'Northwestern Dr'].sort_values(by='address_number', ascending=False)
first_witness = northwestern_residents.iloc[0]
display(first_witness)

,499
id,14887
name,Morty Schapiro
license_id,118009
address_number,4919
address_street_name,Northwestern Dr
ssn,111564949


In [30]:
# Based on the crime scene report description, the second witness lives on "Franklin Ave" and their name is "Annabel"
#พยานคนที่ 2
annabel_franklin = person[(person['address_street_name'] == 'Franklin Ave') & (person['name'].str.contains('Annabel'))]
display(annabel_franklin)

,id,name,license_id,address_number,address_street_name,ssn
665,16371,Annabel Miller,490173,103,Franklin Ave,318771143


คำให้การของพยาน

In [44]:
# Find interviews with both witnesses
witness_ids = [first_witness['id'], annabel_franklin.iloc[0]['id']]
witness_interviews = interview[interview['person_id'].isin(witness_ids)]
display(witness_interviews)

,person_id,transcript
4988,14887,I heard a gunshot and then saw a man run out. ...
4989,16371,"I saw the murder happen, and I recognized the ..."


In [40]:
# Extract keywords from the transcripts
transcript1 = witness_interviews.iloc[0]['transcript']
transcript2 = witness_interviews.iloc[1]['transcript']

print("คำให้การของพยานคนที่ 1:")
print(transcript1)
print("\nคำให้การของพยานคนที่ 2:")
print(transcript2)

คำให้การของพยานคนที่ 1:
I heard a gunshot and then saw a man run out. He had a "Get Fit Now Gym" bag. The membership number on the bag started with "48Z". Only gold members have those bags. The man got into a car with a plate that included "H42W".

คำให้การของพยานคนที่ 2:
I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th.


## หาฆาตกร

In [41]:
# Filter get_fit_now_member based on the membership id
gym_members = fit_member[fit_member['id'].str.startswith('48Z')]
display(gym_members)

,id,person_id,name,membership_start_date,membership_status
154,48Z38,49550,Tomas Baisley,20170203,silver
181,48Z7A,28819,Joe Germuska,20160305,gold
182,48Z55,67318,Jeremy Bowers,20160101,gold


In [42]:
# Filter drivers_license based on the plate number
drivers_with_plate = drivers[drivers['plate_number'].str.contains('H42W')]
display(drivers_with_plate)

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
915,183779,21,65,blue,blonde,female,H42W0X,Toyota,Prius
3529,423327,30,70,brown,brown,male,0H42W2,Chevrolet,Spark LS
6240,664760,21,71,black,black,male,4H42WR,Nissan,Altima


In [37]:
# Filter gym check-in data for January 9th, 2018
gym_checkins_jan9 = fit_checkin[fit_checkin['check_in_date'] == 20180109]
display(gym_checkins_jan9)

,membership_id,check_in_date,check_in_time,check_out_time
113,X0643,20180109,957,1164
889,UK1F2,20180109,344,518
1329,XTE42,20180109,486,1124
1446,1AE2H,20180109,461,944
1703,6LSTG,20180109,399,515
2137,7MWHJ,20180109,273,885
2435,GE5Q8,20180109,367,959
2700,48Z7A,20180109,1600,1730
2701,48Z55,20180109,1530,1700
2702,90081,20180109,1600,1700


In [78]:
# # 1) เลือก Gold member ที่ id เริ่มด้วย 48Z
# gold_members = fit_member[(fit_member['id'].astype(str).str.startswith("48Z")) &
#                           (fit_member['membership_status'] == "gold")]
# print("Gold Members:")
# print(gold_members)

# # 2) join กับ person
# gold_with_person = gold_members.merge(person, left_on="person_id", right_on="id", how="left")
# print("\nGold Members with Person Info:")
# print(gold_with_person)

# # 3) join ต่อกับ drivers_license โดยใช้ license_id
# gold_with_license = gold_with_person.merge(drivers, left_on="license_id", right_on="id", how="left")
# print("\nGold Members with License Info:")
# print(gold_with_license)


# # 4) กรองทะเบียนที่มี "H42W" โดยจัดการค่าว่าง
# suspects = gold_with_license[gold_with_license['plate_number'].notna() & gold_with_license['plate_number'].str.contains("H42W")]
# print("\nSuspects based on Plate Number:")
# print(suspects[['name_x', 'plate_number', 'membership_status', 'id_x']])  # id_x = membership_id


# # 5) Cross-check กับ check-in วันที่ 20180109
# checkins = fit_checkin[(fit_checkin['check_in_date'] == 20180109) &
#                        (fit_checkin['membership_id'].isin(suspects['id_x']))]
# print("\nCheck-ins on 20180109 for Suspects:")
# print(checkins)

# 6) หา suspect ตัวจริง
murderer = suspects.merge(checkins, left_on="id_x", right_on="membership_id", how="inner")
print("\nฆาตรกร คือ:")
print(murderer[['name_x', 'plate_number', 'membership_status']])


ฆาตรกร คือ:
          name_x plate_number membership_status
0  Jeremy Bowers       0H42W2              gold


In [57]:
# Find Jeremy Bowers' interview
jeremy_interview = interview[interview['person_id'] == murderer.iloc[0]['person_id']]
display(jeremy_interview)

,person_id,transcript
4990,67318,I was hired by a woman with a lot of money. I ...


In [65]:
transcript_Jermy = jeremy_interview.iloc[0]['transcript']
print("คำให้การของ Jeremy:")
print(transcript_Jermy)

คำให้การของ Jeremy:
I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5" (65") or 5'7" (67"). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017.



## หาผู้บงการ

Based on Jeremy Bowers' interview, the mastermind is a wealthy woman, around 5'5" to 5'7" tall, with red hair, drives a Tesla Model S, and attended the SQL Symphony Concert 3 times in December 2017.

Let's use these clues to find her.

In [66]:
# Filter persons based on gender, height, and hair color
potential_masterminds = person.merge(drivers, left_on='license_id', right_on='id')
potential_masterminds = potential_masterminds[(potential_masterminds['gender'] == 'female') &
                                             (potential_masterminds['height'].between(65, 67)) &
                                             (potential_masterminds['hair_color'] == 'red')]
display(potential_masterminds)

,id_x,name,license_id,address_number,address_street_name,ssn,id_y,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
80,10704,Shanta Finwall,121750,323,N Kingsway Way,392177155,121750,51,67,amber,red,female,555AGV,Volkswagen,Phaeton
441,14324,Gerry Lofthouse,704869,117,Yarrunga Rd,343814539,704869,76,67,amber,red,female,ZLL18E,Audi,A3
539,15316,Katherine Depolis,609574,2143,Sally Dr,744460595,609574,57,65,blue,red,female,H02637,Mazda,Mazda6
581,15652,Mariah Hatlee,379396,1970,Wirralee Way,246503550,379396,42,67,black,red,female,74FJEW,Mazda,Millenia
1023,19428,Isela Rambo,181202,3620,Back Primrose St,994826376,181202,82,65,amber,red,female,20LE34,Chevrolet,Express 2500
1226,21284,Wanda Bresnan,804587,2319,Omeo Rd,858305461,804587,21,65,black,red,female,3SKZ8L,Lexus,IS F
1489,23593,Nickie Damme,702457,1966,Rattlesnake St,282357434,702457,75,65,amber,red,female,XCT8CN,Toyota,Camry Hybrid
1697,25687,Wanetta Mangat,263558,3836,S Syndicate St,515781105,263558,32,65,green,red,female,SNAO0S,BMW,X5 M
1700,25710,Fransisca Brenda,509142,256,Roesner Blvd,823053012,509142,74,65,blue,red,female,37511O,BMW,750
2009,28553,Bernetta Bang,315095,3163,Raybel Rd,114470474,315095,55,67,blue,red,female,168XV7,BMW,7 Series


In [67]:
# Filter based on car make and model (Tesla Model S)
tesla_drivers = drivers[(drivers['car_make'] == 'Tesla') & (drivers['car_model'] == 'Model S')]
display(tesla_drivers)

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
1105,202298,68,66,green,red,female,500123,Tesla,Model S
2054,291182,65,66,blue,red,female,08CM64,Tesla,Model S
4012,467669,70,65,green,white,male,W528W7,Tesla,Model S
5975,638311,72,63,amber,white,male,0M846T,Tesla,Model S
6428,682231,51,80,green,grey,female,3Z0M62,Tesla,Model S
7083,736081,79,69,brown,red,male,GCAQ6Y,Tesla,Model S
9078,918773,48,65,black,red,female,917UU3,Tesla,Model S


In [68]:
# Find potential masterminds who drive a Tesla Model S
mastermind_candidates = potential_masterminds.merge(tesla_drivers, left_on='license_id', right_on='id')
display(mastermind_candidates)

,id_x,name,license_id,address_number,address_street_name,ssn,id_y,age_x,height_x,eye_color_x,...,car_model_x,id,age_y,height_y,eye_color_y,hair_color_y,gender_y,plate_number_y,car_make_y,car_model_y
0,78881,Red Korb,918773,107,Camerata Dr,961388910,918773,48,65,black,...,Model S,918773,48,65,black,red,female,917UU3,Tesla,Model S
1,90700,Regina George,291182,332,Maple Ave,337169072,291182,65,66,blue,...,Model S,291182,65,66,blue,red,female,08CM64,Tesla,Model S
2,99716,Miranda Priestly,202298,1883,Golden Ave,987756388,202298,68,66,green,...,Model S,202298,68,66,green,red,female,500123,Tesla,Model S


In [69]:
# Filter facebook check-ins for SQL Symphony Concert in December 2017
concert_checkins = fb_checkin[(fb_checkin['event_name'] == 'SQL Symphony Concert') &
                              (fb_checkin['date'].between(20171201, 20171231))]
display(concert_checkins)

,person_id,event_id,event_name,date
4087,62596,1143,SQL Symphony Concert,20171225
4392,19260,1143,SQL Symphony Concert,20171214
6552,58898,1143,SQL Symphony Concert,20171220
6620,69699,1143,SQL Symphony Concert,20171214
6717,19292,1143,SQL Symphony Concert,20171213
7266,43366,1143,SQL Symphony Concert,20171207
8395,92343,1143,SQL Symphony Concert,20171212
10909,28582,1143,SQL Symphony Concert,20171220
10910,28582,1143,SQL Symphony Concert,20171215
13719,81526,1143,SQL Symphony Concert,20171202


In [70]:
# Count how many times each person attended the concert in December 2017
concert_attendance_counts = concert_checkins['person_id'].value_counts().reset_index()
concert_attendance_counts.columns = ['person_id', 'attendance_count']
display(concert_attendance_counts)

,person_id,attendance_count
0,24556,3
1,99716,3
2,28582,2
3,62596,1
4,69699,1
5,19292,1
6,58898,1
7,19260,1
8,92343,1
9,43366,1


In [71]:
# Find the person who attended the concert 3 times
attended_3_times = concert_attendance_counts[concert_attendance_counts['attendance_count'] == 3]
display(attended_3_times)

,person_id,attendance_count
0,24556,3
1,99716,3


In [73]:
# Find the mastermind by merging mastermind candidates with concert attendance
mastermind = mastermind_candidates.merge(attended_3_times, left_on='id_x', right_on='person_id')
display(mastermind)

,id_x,name,license_id,address_number,address_street_name,ssn,id_y,age_x,height_x,eye_color_x,...,age_y,height_y,eye_color_y,hair_color_y,gender_y,plate_number_y,car_make_y,car_model_y,person_id,attendance_count
0,99716,Miranda Priestly,202298,1883,Golden Ave,987756388,202298,68,66,green,...,68,66,green,red,female,500123,Tesla,Model S,99716,3


## สรุป

In [76]:
print("ฆาตกร คือ", murderer.iloc[0]['name_x'])
print("ผู้บงการ คือ", mastermind.iloc[0]['name'])

ฆาตกร คือ Jeremy Bowers
ผู้บงการ คือ Miranda Priestly
